# 3.7 학습 및 검증 실시

- PSPNet 학습 및 검증을 실시합니다. AWS의 GPU 머신으로 계산합니다.
- p2.xlarge에서는 약 12시간이 걸립니다.


# 학습 목표

1.	PSPNet 학습 및 검증을 구현할 수 있다
2.	시맨틱 분할의 파인 튜닝을 이해한다
3.  스케줄러로 epoch마다 학습률을 변화시키는 기법을 구현할 수 있다


# 사전 준비

- 도서의 지시에 따라, 학습된 모델 파일 "pspnet50_ADE20K.pth"을 다운로드하여 "weights" 폴더를 준비합니다.

In [ ]:
# 패키지 import
import random
import math
import time
import pandas as pd
import numpy as np

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# 초기설정
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# DataLoader 작성

In [ ]:
from utils.dataloader import make_datapath_list, DataTransform, VOCDataset

# 파일 경로 리스트 작성
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath=rootpath)

# Dataset 작성
# (RGB) 색의 평균값과 표준편차
color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

# DataLoader 작성
batch_size = 8

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 사전형 변수로 정리
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


# 네트워크 모델 작성

In [ ]:
from utils.pspnet import PSPNet

# 파인 튜닝으로 PSPNet을 작성
# ADE20K 데이터 세트의 학습된 모델을 사용하며, ADE20K는 클래스 수가 150입니다
net = PSPNet(n_classes=150)

# ADE20K 학습된 파라미터를 읽기
state_dict = torch.load("./weights/pspnet50_ADE20K.pth")
net.load_state_dict(state_dict)

# 분류용의 합성곱층을, 출력수 21으로 바꿈
n_classes = 21
net.decode_feature.classification = nn.Conv2d(
    in_channels=512, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

net.aux.classification = nn.Conv2d(
    in_channels=256, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

# 교체한 합성곱층을 초기화한다. 활성화 함수는 시그모이드 함수이므로 Xavier를 사용한다.
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight.data)
        if m.bias is not None:  # 바이어스 항이 있는 경우
            nn.init.constant_(m.bias, 0.0)

net.decode_feature.classification.apply(weights_init)
net.aux.classification.apply(weights_init)

print('네트워크 설정 완료: 학습된 가중치를 로드했습니다')

네트워크 설정 완료 : 학습된 가중치를 로드했습니다


In [ ]:
net

PSPNet(
  (feature_conv): FeatureMap_convolution(
    (cbnr_1): conv2DBatchNormRelu(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
    )
    (cbnr_2): conv2DBatchNormRelu(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
    )
    (cbnr_3): conv2DBatchNormRelu(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (feature_res_1): ResidualBlockPSP(
    (block1): bottleNeckPSP(
      (cb

# 손실함수 정의

In [ ]:
# 손실함수 정의
class PSPLoss(nn.Module):
    """PSPNet의 손실함수 클래스입니다"""

    def __init__(self, aux_weight=0.4):
        super(PSPLoss, self).__init__()
        self.aux_weight = aux_weight  # aux_loss의 가중치

    def forward(self, outputs, targets):
        """
        손실함수 계산

        Parameters
        ----------
        outputs : PSPNet의 출력(tuple)
            (output=torch.Size([num_batch, 21, 475, 475]), output_aux=torch.Size([num_batch, 21, 475, 475]))。

        targets : [num_batch, 475, 475]
            정답 어노테이션 정보

        Returns
        -------
        loss : 텐서
            손실값
        """

        loss = F.cross_entropy(outputs[0], targets, reduction='mean')
        loss_aux = F.cross_entropy(outputs[1], targets, reduction='mean')

        return loss+self.aux_weight*loss_aux


criterion = PSPLoss(aux_weight=0.4)


# 최적화 기법 설정

In [ ]:
# 파인 튜닝이므로, 학습률은 작게
optimizer = optim.SGD([
    {'params': net.feature_conv.parameters(), 'lr': 1e-3},
    {'params': net.feature_res_1.parameters(), 'lr': 1e-3},
    {'params': net.feature_res_2.parameters(), 'lr': 1e-3},
    {'params': net.feature_dilated_res_1.parameters(), 'lr': 1e-3},
    {'params': net.feature_dilated_res_2.parameters(), 'lr': 1e-3},
    {'params': net.pyramid_pooling.parameters(), 'lr': 1e-3},
    {'params': net.decode_feature.parameters(), 'lr': 1e-2},
    {'params': net.aux.parameters(), 'lr': 1e-2},
], momentum=0.9, weight_decay=0.0001)


# 스케쥴러 설정
def lambda_epoch(epoch):
    max_epoch = 30
    return math.pow((1-epoch/max_epoch), 0.9)


scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_epoch)


# 학습 및 검증 실시

In [ ]:
# 모델을 학습시키는 함수를 작성
def train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epochs):

    # GPU가 사용 가능한지 확인
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치: ", device)

    # 네트워크를 GPU로
    net.to(device)

    # 네트워크가 어느 정도 고정되면 고속화한다
    torch.backends.cudnn.benchmark = True

    # 화상의 매수
    num_train_imgs = len(dataloaders_dict["train"].dataset)
    num_val_imgs = len(dataloaders_dict["val"].dataset)
    batch_size = dataloaders_dict["train"].batch_size

    # 반복자의 카운터 설정
    iteration = 1
    logs = []

    # multiple minibatch
    batch_multiplier = 3

    # epoch 루프
    for epoch in range(num_epochs):

        # 시작 시간 저장
        t_epoch_start = time.time()
        t_iter_start = time.time()
        epoch_train_loss = 0.0  # epoch의 손실합
        epoch_val_loss = 0.0  # epoch의 손실합

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epoch별 훈련 및 검증 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 모델을 훈련 모드로
                scheduler.step()  # 최적화 scheduler 갱신
                optimizer.zero_grad()
                print('(train)')

            else:
                if((epoch+1) % 5 == 0):
                    net.eval()   # 모델을 검증 모드로
                    print('-------------')
                    print('(val)')
                else:
                    # 검증은 다섯 번 중에 한 번만 수행
                    continue

            # 데이터 로더에서 minibatch씩 꺼내 루프
            count = 0  # multiple minibatch
            for imges, anno_class_imges in dataloaders_dict[phase]:
                # 미니배치 크기가 1이면 배치 노멀라이제이션에서 오류가 발생하므로 회피
                if imges.size()[0] == 1:
                    continue

                # GPU가 사용가능하면 GPU에 데이터를 보낸다
                imges = imges.to(device)
                anno_class_imges = anno_class_imges.to(device)

                # multiple minibatch로 파라미터 갱신
                if (phase == 'train') and (count == 0):
                    optimizer.step()
                    optimizer.zero_grad()
                    count = batch_multiplier

                # 순전파(forward) 계산
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(imges)
                    loss = criterion(
                        outputs, anno_class_imges.long()) / batch_multiplier

                    # 훈련시에는 역전파
                    if phase == 'train':
                        loss.backward()  # 경사 계산
                        count -= 1  # multiple minibatch

                        if (iteration % 10 == 0):  # 10iter에 한 번, loss를 표시
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('반복 {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item()/batch_size*batch_multiplier, duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item() * batch_multiplier
                        iteration += 1

                    # 검증 시
                    else:
                        epoch_val_loss += loss.item() * batch_multiplier

        # epoch의 phase별 loss와 정답률
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss/num_train_imgs, epoch_val_loss/num_val_imgs))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # 로그 저장
        log_epoch = {'epoch': epoch+1, 'train_loss': epoch_train_loss /
                     num_train_imgs, 'val_loss': epoch_val_loss/num_val_imgs}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

    # 최후의 네트워크를 저장
    torch.save(net.state_dict(), 'weights/pspnet50_' +
               str(epoch+1) + '.pth')


In [ ]:
# 학습 및 검증 실행
num_epochs = 30
train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epochs=num_epochs)


사용 장치: cuda:0
-------------
Epoch 1/30
-------------
(train)
반복 10 || Loss: 0.3835 || 10iter: 83.2019 sec.
반복 20 || Loss: 0.2189 || 10iter: 50.9118 sec.
반복 30 || Loss: 0.1510 || 10iter: 50.8032 sec.
반복 40 || Loss: 0.1658 || 10iter: 50.7695 sec.
반복 50 || Loss: 0.0886 || 10iter: 50.6645 sec.
반복 60 || Loss: 0.0728 || 10iter: 50.6198 sec.
반복 70 || Loss: 0.1165 || 10iter: 50.9016 sec.
반복 80 || Loss: 0.1351 || 10iter: 50.4392 sec.
반복 90 || Loss: 0.2174 || 10iter: 50.6154 sec.
반복 100 || Loss: 0.0904 || 10iter: 50.5267 sec.
반복 110 || Loss: 0.1408 || 10iter: 50.4316 sec.
반복 120 || Loss: 0.0668 || 10iter: 50.5083 sec.
반복 130 || Loss: 0.1251 || 10iter: 50.7642 sec.
반복 140 || Loss: 0.1467 || 10iter: 50.5163 sec.
반복 150 || Loss: 0.0794 || 10iter: 50.5443 sec.
반복 160 || Loss: 0.1435 || 10iter: 50.4825 sec.
반복 170 || Loss: 0.2098 || 10iter: 50.4970 sec.
반복 180 || Loss: 0.1435 || 10iter: 50.5844 sec.
-------------
epoch 1 || Epoch_TRAIN_Loss:0.1771 ||Epoch_VAL_Loss:0.0000
timer:  1056.2730 sec.
------

반복 1380 || Loss: 0.0715 || 10iter: 50.8677 sec.
반복 1390 || Loss: 0.0787 || 10iter: 50.8265 sec.
반복 1400 || Loss: 0.0590 || 10iter: 50.7918 sec.
반복 1410 || Loss: 0.0693 || 10iter: 50.6000 sec.
반복 1420 || Loss: 0.0749 || 10iter: 50.6794 sec.
반복 1430 || Loss: 0.1086 || 10iter: 50.6710 sec.
반복 1440 || Loss: 0.0649 || 10iter: 50.8908 sec.
반복 1450 || Loss: 0.0552 || 10iter: 50.7711 sec.
반복 1460 || Loss: 0.0374 || 10iter: 50.5102 sec.
-------------
epoch 8 || Epoch_TRAIN_Loss:0.0574 ||Epoch_VAL_Loss:0.0000
timer:  1026.2220 sec.
-------------
Epoch 9/30
-------------
(train)
반복 1470 || Loss: 0.0475 || 10iter: 28.4563 sec.
반복 1480 || Loss: 0.0209 || 10iter: 50.8128 sec.
반복 1490 || Loss: 0.0394 || 10iter: 50.5409 sec.
반복 1500 || Loss: 0.0464 || 10iter: 50.7096 sec.
반복 1510 || Loss: 0.0371 || 10iter: 50.8713 sec.
반복 1520 || Loss: 0.0364 || 10iter: 50.9948 sec.
반복 1530 || Loss: 0.1000 || 10iter: 50.6096 sec.
반복 1540 || Loss: 0.0512 || 10iter: 50.6501 sec.
반복 1550 || Loss: 0.0871 || 10iter: 50.572

반복 2740 || Loss: 0.0271 || 10iter: 50.7591 sec.
-------------
(val)
-------------
epoch 15 || Epoch_TRAIN_Loss:0.0459 ||Epoch_VAL_Loss:0.0714
timer:  1347.2690 sec.
-------------
Epoch 16/30
-------------
(train)
반복 2750 || Loss: 0.0726 || 10iter: 22.7602 sec.
반복 2760 || Loss: 0.0297 || 10iter: 50.7317 sec.
반복 2770 || Loss: 0.0294 || 10iter: 50.7034 sec.
반복 2780 || Loss: 0.0386 || 10iter: 50.9082 sec.
반복 2790 || Loss: 0.0381 || 10iter: 50.8296 sec.
반복 2800 || Loss: 0.0466 || 10iter: 50.6776 sec.
반복 2810 || Loss: 0.0318 || 10iter: 50.8160 sec.
반복 2820 || Loss: 0.0314 || 10iter: 50.5198 sec.
반복 2830 || Loss: 0.0482 || 10iter: 50.7523 sec.
반복 2840 || Loss: 0.0258 || 10iter: 50.8890 sec.
반복 2850 || Loss: 0.0436 || 10iter: 50.7661 sec.
반복 2860 || Loss: 0.0503 || 10iter: 50.5520 sec.
반복 2870 || Loss: 0.0293 || 10iter: 50.5255 sec.
반복 2880 || Loss: 0.0287 || 10iter: 50.4744 sec.
반복 2890 || Loss: 0.0360 || 10iter: 50.7980 sec.
반복 2900 || Loss: 0.0362 || 10iter: 50.8190 sec.
반복 2910 || Loss: 0.

반복 4060 || Loss: 0.0297 || 10iter: 50.8080 sec.
반복 4070 || Loss: 0.0244 || 10iter: 50.4723 sec.
반복 4080 || Loss: 0.0669 || 10iter: 50.6491 sec.
반복 4090 || Loss: 0.0236 || 10iter: 50.5933 sec.
반복 4100 || Loss: 0.0330 || 10iter: 50.7665 sec.
반복 4110 || Loss: 0.0284 || 10iter: 50.5859 sec.
반복 4120 || Loss: 0.0409 || 10iter: 50.5714 sec.
반복 4130 || Loss: 0.0505 || 10iter: 50.9581 sec.
반복 4140 || Loss: 0.0372 || 10iter: 50.7392 sec.
반복 4150 || Loss: 0.0457 || 10iter: 50.8464 sec.
반복 4160 || Loss: 0.0450 || 10iter: 50.8673 sec.
반복 4170 || Loss: 0.0761 || 10iter: 50.6163 sec.
반복 4180 || Loss: 0.0455 || 10iter: 50.8787 sec.
반복 4190 || Loss: 0.0393 || 10iter: 50.5992 sec.
반복 4200 || Loss: 0.0424 || 10iter: 50.7738 sec.
-------------
epoch 23 || Epoch_TRAIN_Loss:0.0423 ||Epoch_VAL_Loss:0.0000
timer:  1025.3780 sec.
-------------
Epoch 24/30
-------------
(train)
반복 4210 || Loss: 0.0193 || 10iter: 0.2901 sec.
반복 4220 || Loss: 0.0379 || 10iter: 50.4680 sec.
반복 4230 || Loss: 0.0544 || 10iter: 50.53

반복 5420 || Loss: 0.0194 || 10iter: 50.6732 sec.
반복 5430 || Loss: 0.0428 || 10iter: 50.8644 sec.
반복 5440 || Loss: 0.0494 || 10iter: 50.8474 sec.
반복 5450 || Loss: 0.0507 || 10iter: 50.8454 sec.
반복 5460 || Loss: 0.0448 || 10iter: 51.0749 sec.
반복 5470 || Loss: 0.0192 || 10iter: 51.0534 sec.
반복 5480 || Loss: 0.0663 || 10iter: 50.8508 sec.
반복 5490 || Loss: 0.0642 || 10iter: 50.8363 sec.
-------------
(val)
-------------
epoch 30 || Epoch_TRAIN_Loss:0.0399 ||Epoch_VAL_Loss:0.0701
timer:  1352.9696 sec.


끝